In [1]:
import re
import random
import time

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from constants import PROVINCIAS
from data.io import save_df_to_parquet
from scrapper import IdealistaScraper
from data.normalizator import NormalizarDataFrame

In [ ]:
pd.options.display.max_columns = None

# Extracción de los datos Raw

In [ ]:
id_pisos_comunidad_autonoma = []

for provincia in PROVINCIAS:
    # Definir los headers
    headers_provincia = {
        "User-Agent": UserAgent().random,
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Upgrade-Insecure-Requests": "1",
        "Referer": "https://www.google.com/",
    }

    for pag in range(1, 7):
        # Enlace relacionado con todos los pisos en renta para la provincia de interés.
        # Solo resultados de la primera página.
        url_provincia = f"https://www.idealista.com/alquiler-viviendas/{provincia}-provincia/pagina-{str(pag)}.htm"

        # Enviar la petición y ver la respuesta
        response_provincia = requests.get(url_provincia, headers=headers_provincia)

        if response_provincia.status_code == 200:
            # Analizar el contenido HTML usando BeautifulSoup
            soup = BeautifulSoup(response_provincia.content, "html.parser")

            # Extraer los tags que indican los id de los pisos publicados
            articles = soup.find_all("article", attrs={"data-element-id": True})
            id_pisos_publicados_provincia = [
                article["data-element-id"] for article in articles
            ]

            # Extender la lista de ids
            id_pisos_comunidad_autonoma.extend(id_pisos_publicados_provincia)

            # Agregar un delay para imitar un comportamiento humano
            time.sleep(random.uniform(1, 3))
            print(f"Scrapping de ids para la provincia de: {provincia} página {pag}")
        else:
            raise Exception(
                f"Scrapping fallido con código de respuesta {provincia} página {pag}"
                f"para la provincia: {provincia}"
            )

In [ ]:
print("Número de pisos para hacer scrapping:", len(id_pisos_comunidad_autonoma))

In [ ]:
info_pisos = []
contador = 0

# Extracción de la info de todos los pisos en la primera página
for piso in id_pisos_comunidad_autonoma:

    # Definir la url según el id
    url_piso = f"https://www.idealista.com/inmueble/{piso}/"

    # Generar
    scraper = IdealistaScraper(url_piso)
    data_piso = scraper.scrape()

    # Agregar el id del piso
    data_piso["id_url"] = piso

    # Guardar los resultados en la lista
    info_pisos.append(data_piso)

    # Generar un delay para imitar una búsqueda humana
    time.sleep(random.uniform(2, 5))

    # Aumentar el contador en 1
    contador += 1

    # Dormir la ejecución 15 segundos cuando se haya llegado
    # a un múltiplo de 100 peticiones
    if contador % 100 == 0:
        print(f"Se ha llegado a un múltiplo de 100 (durmiendo peticiones): {contador}")
        time.sleep(30)

In [ ]:
# Convertir la lista de diccionarios a DataFrame
dataframe_info_pisos = pd.DataFrame(info_pisos)
dataframe_info_pisos

In [ ]:
# Guardar el DataFrame
save_df_to_parquet(dataframe_info_pisos, "../data/raw/", "pisos", replace=True)

# Limpieza de los datos a normalizar

In [2]:
# Leer los datos en raw
dataframe_info_pisos = pd.read_parquet("../data/raw/pisos.parquet")

# Rename (cambio temporal)
dataframe_info_pisos.rename(
    columns={"caracteristicas_básicas": "caracteristicas_basicas"}, inplace=True
)
dataframe_info_pisos.head()

,title,referencia_anuncio,price,info_features,caracteristicas_basicas,certificado_energetico,Precio del inmueble:,Precio por m²:,ubicacion,id_url
0,Alquiler de Chalet adosado en Urbanització Deu...,ll-1,1.800,"[293 m², 5 hab., Garaje incluido, Garaje inclu...","[Chalet adosado, 293 m² construidos, 228 m² út...","[{'Consumo:': ['', 'icon-energy-c-g'], 'Emisio...",1.800 €/mes,"6,14 €/m²","[Urbanització Deulofeu, 3 -1, Puigcerdà, La Ce...",102140869
1,Alquiler de Piso en calle de la Font de la Teu...,105322130,1.980,"[127 m², 4 hab., Planta 2ª exterior con ascens...","[127 m² construidos, 4 habitaciones, 2 baños, ...","[{'Consumo:': ['32 kWh/m² año', 'icon-energy-c...",1.980 €/mes,"15,59 €/m²","[Calle de la Font de la Teula, 14, Distrito Fo...",105322130
2,"Alquiler de Piso en Can Sabata - Mas Baell, Ll...",104525369,1.600,"[60 m², 1 hab., Planta 5ª exterior con ascensor]","[60 m² construidos, 55 m² útiles, 1 habitación...","[{'Consumo:': ['', 'icon-energy-c-a'], 'Emisio...",1.600 €/mes,"26,67 €/m²","[Distrito Can Sabata - Mas Baell, Lloret de Ma...",104525369
3,Alquiler de Casa de pueblo en calle Sant Gaiet...,7046,1.400,"[210 m², 4 hab., Garaje incluido, Garaje inclu...","[Casa de pueblo, 3 plantas, 210 m² construidos...","[{'Consumo:': ['94 kWh/m² año', 'icon-energy-c...",1.400 €/mes,"6,67 €/m²","[Calle Sant Gaietà, 5, Boadella d'Empordà, Alt...",105113238
4,Alquiler de Casa o chalet independiente en cal...,104544516,2.000,"[230 m², 3 hab.]","[Casa o chalet independiente, 2 plantas, 230 m...","[{'Consumo:': ['', 'icon-energy-c-e'], 'Emisio...",2.000 €/mes,"8,70 €/m²","[Calle de Carles Riba s/n, Distrito Residencia...",104544516


In [3]:
dataframe_info_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   title                    468 non-null    object
 1   referencia_anuncio       468 non-null    object
 2   price                    468 non-null    object
 3   info_features            468 non-null    object
 4   caracteristicas_basicas  468 non-null    object
 5   certificado_energetico   468 non-null    object
 6   Precio del inmueble:     468 non-null    object
 7   Precio por m²:           468 non-null    object
 8   ubicacion                468 non-null    object
 9   id_url                   468 non-null    object
dtypes: object(10)
memory usage: 36.7+ KB


In [11]:
normalizador = NormalizarDataFrame(dataframe_info_pisos)
dataframe_info_pisos_normalizado = normalizador.normalize_dataframe()
save_df_to_parquet(
    dataframe_info_pisos_normalizado, "../data/normalized/", "pisos", replace=True
)

In [10]:
dataframe_info_pisos_normalizado.head()

,title,referencia_anuncio,price,precio_inmueble,precio_m2,ubicacion,id_url,superficie_m2,habitaciones,garaje,planta,ascensor,consumo_energetico_valor,consumo_energetico_icono,emisiones_valor,emisiones_icono,caracteristicas_basicas_superficie_m2,superficie_util_m2,caracteristicas_basicas_habitaciones,caracteristicas_basicas_banos,parcela_m2,terraza,balcon,garaje_incluido,segunda_mano_buen_estado,armarios_empotrados,orientacion,cocina_equipada,amueblada,calefaccion,trastero,construccion,plantas
0,Alquiler de Chalet adosado en Urbanització Deu...,ll-1,1800.0,1800,6.14,"[Urbanització Deulofeu, 3 -1, Puigcerdà, La Ce...",102140869,293,5.0,True,NaN,None,None,icon-energy-c-g,None,icon-energy-c-g,293,228.0,5.0,4,200.0,True,False,True,True,True,sur,True,True,individual: gas natural,False,NaN,NaN
1,Alquiler de Piso en calle de la Font de la Teu...,105322130,1980.0,1980,15.59,"[Calle de la Font de la Teula, 14, Distrito Fo...",105322130,127,4.0,True,2.0,True,32 kWh/m² año,icon-energy-c-a,5 kg CO2/m² año,icon-energy-c-a,127,NaN,4.0,2,NaN,True,True,True,True,True,"sur, oeste",True,True,individual,True,2024.0,NaN
2,"Alquiler de Piso en Can Sabata - Mas Baell, Ll...",104525369,1600.0,1600,26.67,"[Distrito Can Sabata - Mas Baell, Lloret de Ma...",104525369,60,1.0,False,5.0,True,None,icon-energy-c-a,None,icon-energy-c-a,60,55.0,1.0,1,NaN,True,False,False,True,True,None,True,True,None,False,NaN,NaN
3,Alquiler de Casa de pueblo en calle Sant Gaiet...,7046,1400.0,1400,6.67,"[Calle Sant Gaietà, 5, Boadella d'Empordà, Alt...",105113238,210,4.0,True,NaN,None,94 kWh/m² año,icon-energy-c-c,16 kg CO2/m² año,icon-energy-c-c,210,186.0,4.0,4,NaN,True,False,True,True,True,"sur, este",True,True,individual,True,1840.0,3.0
4,Alquiler de Casa o chalet independiente en cal...,104544516,2000.0,2000,8.70,"[Calle de Carles Riba s/n, Distrito Residencia...",104544516,230,3.0,False,NaN,None,None,icon-energy-c-e,None,icon-energy-c-e,230,227.0,3.0,3,869.0,True,False,False,True,False,"norte, sur, este, oeste",True,True,central,False,2004.0,2.0
